<a href="https://colab.research.google.com/github/Arnav-1Y/Machine-Learning/blob/main/Copy_of_Student_DistractedDrivers_AppDeployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deploying an App with Streamlit


![](https://raw.githubusercontent.com/NolanChai/model_repo/main/Distracted_Driving.jpg)

Congratulations for making it this far in the project! So far, we have:
- Trained a computer vision model to detect whether or not a driver is attentive to the road
- Generated saliency maps for our predictions
- Fine tuned expert models via transfer learning

In this notebook, we will be:
- Deploying our AI model on Streamlit
- Customizing the user interface!

# Part 1. Streamlit - Deploying your model to the web
Today we will be using Streamlit, a framework to easily build web applications, to deploy our models to the web so that they can be shared to the web!

Take a moment to look through examples of websites built with Streamlit [here](https://streamlit.io/gallery?category=favorites). As a class, choose your favorite and answer the following **questions:**
* Who is this application for?
* How does the user input data - are these intuitive ways of interacting with the app?
* What does the application do with the data?
* Evaluate the ease of use and look of the application.

Now that we’ve seen what is possible with Streamlit, let’s try to deploy our **Distracted Drivers model** to the web!

First, let's make sure that we are using our GPU! <br> This block of code should output `Found GPU at: /device:GPU:0`. <br> If not, go to your `Runtime` tab, and `Change Runtime`.
- Select a T4 GPU or TPU under the Hardware Accelerator section!

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('No GPU Found! :(')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!sudo pip install tensorflow keras
# or install with the optional dependency group
!sudo pip install scikeras[tensorflow]
import tensorflow as tf
import keras
#!sudo pip install
#!sudo pip install keras
#!sudo pip install tensorflow[and-cuda]
#print(tf.__version__)
#print(keras.__version__)

In [ ]:
#@title Run this to set up! (This may take 2-3 minutes)

# Hide Warnings -------------------------------------------
import os
import sys
import warnings

print(sys.version)
warnings.filterwarnings('ignore')

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Installations --------------------------------------------
with HiddenPrints():
    #!pip update -y
    #!pip install python3.12 python3.12-distutils -y
    !sudo pip -q install streamlit
    !sudo pip -q install pyngrok
    # Uninstall existing numpy that might conflict, then install compatible version for imgaug
    !sudo pip uninstall numpy -y
    !sudo pip install numpy==1.26.6
    !sudo pip install imgaug
    # Install core ML libraries, which should now respect the installed numpy
    !sudo pip install tensorflow keras
# or install with the optional dependency group
    !sudo pip install scikeras[tensorflow]
    # Add installation for tf-keras-vis, which is used for saliency map
    !sudo pip install tf-keras-vis
    !wget -q --show-progress 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Driver%20Distraction%20Detection/metadata.csv'
    !wget -q --show-progress 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Driver%20Distraction%20Detection/image_data.npy'
    # Want to move these to a bucket
    !wget https://huggingface.co/NolanChai/Inspirit_Expert_Models/resolve/main/DenseNet121.h5
    !wget https://huggingface.co/NolanChai/Inspirit_Expert_Models/resolve/main/ResNet50.h5
    !wget https://huggingface.co/NolanChai/Inspirit_Expert_Models/resolve/main/vgg16.h5
    !wget https://huggingface.co/NolanChai/Inspirit_Expert_Models/resolve/main/vgg19.h5

# Imports --------------------------------------------------
import cv2
import numpy as np
import streamlit
import zipfile
import numpy as np
import pandas as pd
from pyngrok import ngrok
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import model_selection
from collections import Counter
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Reshape, Dense, Conv2D, GlobalAveragePooling2D
#!pip install -q git+https://github.com/rdk2132/scikeras # workaround for scikeras deprecation
from scikeras.wrappers import KerasClassifier
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, DenseNet121
from imgaug import augmenters

NP_INT_TYPES = {np.int8, np.int16, np.int32, np.int64}
NP_UINT_TYPES = {np.uint8, np.uint16, np.uint32, np.uint64}
NP_FLOAT_TYPES = {np.float16, np.float32, np.float64}

# Helper Functions ------------------------------------------
def label_to_numpy(labels):
  final_labels = np.zeros((len(labels), 4))
  for i in range(len(labels)):
    label = labels[i]
    if label == 'Attentive':
      final_labels[i,:] = np.array([1, 0, 0, 0])
    if label == 'DrinkingCoffee':
      final_labels[i,:] = np.array([0, 1, 0, 0])
    if label == 'UsingMirror':
      final_labels[i,:] = np.array([0, 0, 1, 0])
    if label == 'UsingRadio':
      final_labels[i,:] = np.array([0, 0, 0, 1])
  return final_labels

def augment(data, augmenter):
  if len(data.shape) == 3:
    return augmenter.augment_image(data)
  if len(data.shape) == 4:
    return augmenter.augment_images(data)

def rotate(data, rotate):
  fun = augmenters.Affine(rotate = rotate)
  return augment(data, fun)

def shear(data, shear):
  fun = augmenters.Affine(shear = shear)
  return augment(data, fun)

def scale(data, scale):
  fun = augmenters.Affine(scale = shear)
  return augment(data, fun)

def flip_left_right(data):
  fun = augmenters.Fliplr()
  return augment(data, fun)

def flip_up_down(data):
  fun = augmenters.Flipud()
  return augment(data, fun)

def remove_color(data, channel):
  new_data = data.copy()
  if len(data.shape) == 3:
    new_data[:,:,channel] = 0
    return new_data
  if len(data.shape) == 4:
    new_data[:,:,:,channel] = 0
    return new_data

class pkg:
  #### DOWNLOADING AND LOADING DATA
  def get_metadata(metadata_path, which_splits = ['train', 'test']):
    '''returns metadata dataframe which contains columns of:
       * index: index of data into numpy data
       * class: class of image
       * split: which dataset split is this a part of?
    '''
    metadata = pd.read_csv(metadata_path)
    keep_idx = metadata['split'].isin(which_splits)
    metadata = metadata[keep_idx]

    # Get dataframes for each class.
    df_coffee_train = metadata[(metadata['class'] == 'DrinkingCoffee') & \
                         (metadata['split'] == 'train')]
    df_coffee_test = metadata[(metadata['class'] == 'DrinkingCoffee') & \
                         (metadata['split'] == 'test')]
    df_mirror_train = metadata[(metadata['class'] == 'UsingMirror') & \
                         (metadata['split'] == 'train')]
    df_mirror_test = metadata[(metadata['class'] == 'UsingMirror') & \
                         (metadata['split'] == 'test')]
    df_attentive_train = metadata[(metadata['class'] == 'Attentive') & \
                         (metadata['split'] == 'train')]
    df_attentive_test = metadata[(metadata['class'] == 'Attentive') & \
                         (metadata['split'] == 'test')]
    df_radio_train = metadata[(metadata['class'] == 'UsingRadio') & \
                         (metadata['split'] == 'train')]
    df_radio_test = metadata[(metadata['class'] == 'UsingRadio') & \
                         (metadata['split'] == 'test')]

    # Get number of items in class with lowest number of images.
    num_samples_train = min(df_coffee_train.shape[0], \
                            df_mirror_train.shape[0], \
                            df_attentive_train.shape[0], \
                            df_radio_train.shape[0])
    num_samples_test = min(df_coffee_test.shape[0], \
                            df_mirror_test.shape[0], \
                            df_attentive_test.shape[0], \
                            df_radio_test.shape[0])

    # Resample each of the classes and concatenate the images.
    metadata_train = pd.concat([df_coffee_train.sample(num_samples_train), \
                          df_mirror_train.sample(num_samples_train), \
                          df_attentive_train.sample(num_samples_train), \
                          df_radio_train.sample(num_samples_train) ])
    metadata_test = pd.concat([df_coffee_test.sample(num_samples_test), \
                          df_mirror_test.sample(num_samples_test), \
                          df_attentive_test.sample(num_samples_test), \
                          df_radio_test.sample(num_samples_test) ])

    metadata = pd.concat( [metadata_train, metadata_test] )

    return metadata

  def get_data_split(split_name, flatten, all_data, metadata, image_shape):
    '''
    returns images (data), labels from folder of format [image_folder]/[split_name]/[class_name]/
    flattens if flatten option is True
    '''
    # Get dataframes for each class.
    df_coffee_train = metadata[(metadata['class'] == 'DrinkingCoffee') & \
                         (metadata['split'] == 'train')]
    df_coffee_test = metadata[(metadata['class'] == 'DrinkingCoffee') & \
                         (metadata['split'] == 'test')]
    df_mirror_train = metadata[(metadata['class'] == 'UsingMirror') & \
                         (metadata['split'] == 'train')]
    df_mirror_test = metadata[(metadata['class'] == 'UsingMirror') & \
                         (metadata['split'] == 'test')]
    df_attentive_train = metadata[(metadata['class'] == 'Attentive') & \
                         (metadata['split'] == 'train')]
    df_attentive_test = metadata[(metadata['class'] == 'Attentive') & \
                         (metadata['split'] == 'test')]
    df_radio_train = metadata[(metadata['class'] == 'UsingRadio') & \
                         (metadata['split'] == 'train')]
    df_radio_test = metadata[(metadata['class'] == 'UsingRadio') & \
                         (metadata['split'] == 'test')]

    # Get number of items in class with lowest number of images.
    num_samples_train = min(df_coffee_train.shape[0], \
                            df_mirror_train.shape[0], \
                            df_attentive_train.shape[0], \
                            df_radio_train.shape[0])
    num_samples_test = min(df_coffee_test.shape[0], \
                            df_mirror_test.shape[0], \
                            df_attentive_test.shape[0], \
                            df_radio_test.shape[0])

    # Resample each of the classes and concatenate the images.
    metadata_train = pd.concat([df_coffee_train.sample(num_samples_train), \
                          df_mirror_train.sample(num_samples_train), \
                          df_attentive_train.sample(num_samples_train), \
                          df_radio_train.sample(num_samples_train) ])
    metadata_test = pd.concat([df_coffee_test.sample(num_samples_test), \
                          df_mirror_test.sample(num_samples_test), \
                          df_attentive_test.sample(num_samples_test), \
                          df_radio_test.sample(num_samples_test) ])

    metadata = pd.concat( [metadata_train, metadata_test] )

    sub_df = metadata[metadata['split'].isin([split_name])]
    index  = sub_df['index'].values
    labels = sub_df['class'].values
    data = all_data[index,:]
    if flatten:
      data = data.reshape([-1, np.product(image_shape)])
    return data, labels

  def get_train_data(flatten, all_data, metadata, image_shape):
    return get_data_split('train', flatten, all_data, metadata, image_shape)

  def get_test_data(flatten, all_data, metadata, image_shape):
    return get_data_split('test', flatten, all_data, metadata, image_shape)

  def get_field_data(flatten, all_data, metadata, image_shape):
    return get_data_split('field', flatten, all_data, metadata, image_shape)

class helpers:
  #### PLOTTING
  def plot_image(data, num_ims, figsize=(8,6), labels = [], index = None, image_shape = [64,64,3]):
    '''
    if data is a single image, display that image

    if data is a 4d stack of images, display that image
    '''
    print(data.shape)
    num_dims   = len(data.shape)
    num_labels = len(labels)

    # reshape data if necessary
    if num_dims == 1:
      data = data.reshape(target_shape)
    if num_dims == 2:
      data = data.reshape(-1,image_shape[0],image_shape[1],image_shape[2])
    num_dims   = len(data.shape)

    # check if single or multiple images
    if num_dims == 3:
      if num_labels > 1:
        print('Multiple labels does not make sense for single image.')
        return

      label = labels
      if num_labels == 0:
        label = ''
      image = data

    if num_dims == 4:
      image = data[index, :]
      label = labels[index]

    # plot image of interest

    nrows=int(np.sqrt(num_ims))
    ncols=int(np.ceil(num_ims/nrows))
    print(nrows,ncols)
    count=0
    if nrows==1 and ncols==1:
      print('Label: %s'%label)
      plt.imshow(image)
      plt.show()
    else:
      print(labels)
      fig = plt.figure(figsize=figsize)
      for i in range(nrows):
        for j in range(ncols):
          if count<num_ims:
            fig.add_subplot(nrows,ncols,count+1)
            plt.imshow(image[count])
            count+=1
      fig.set_size_inches(18.5, 10.5)
      plt.show()



  #### QUERYING AND COMBINING DATA
  def get_misclassified_data(data, labels, predictions):
    '''
    Gets the data and labels that are misclassified in a classification task
    Returns:
    -missed_data
    -missed_labels
    -predicted_labels (corresponding to missed_labels)
    -missed_index (indices of items in original dataset)
    '''
    missed_index     = np.where(np.abs(predictions.squeeze() - labels.squeeze()) > 0)[0]
    missed_labels    = labels[missed_index]
    missed_data      = data[missed_index,:]
    predicted_labels = predictions[missed_index]
    return missed_data, missed_labels, predicted_labels, missed_index

  def combine_data(data_list, labels_list):
    return np.concatenate(data_list, axis = 0), np.concatenate(labels_list, axis = 0)

  def model_to_string(model):
    import re
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))

    for layer in model.layers:
      if hasattr(layer,"activation"):
        stringlist.append(str(layer.activation))

    sms = "\n".join(stringlist)
    sms = re.sub(r'_\d\d\d','', sms)
    sms = re.sub(r'_\d\d','', sms)
    sms = re.sub(r'_\d','', sms)
    return sms

  def plot_acc(history, ax = None, xlabel = 'Epoch #'):
    # i'm sorry for this function's code. i am so sorry.
    history = history.history
    history.update({'epoch':list(range(len(history['val_accuracy'])))})
    history = pd.DataFrame.from_dict(history)

    best_epoch = history.sort_values(by = 'val_accuracy', ascending = False).iloc[0]['epoch']

    if not ax:
      f, ax = plt.subplots(1,1)
    sns.lineplot(x = 'epoch', y = 'val_accuracy', data = history, label = 'Validation', ax = ax)
    sns.lineplot(x = 'epoch', y = 'accuracy', data = history, label = 'Training', ax = ax)
    ax.axhline(0.25, linestyle = '--',color='red', label = 'Chance')
    ax.axvline(x = best_epoch, linestyle = '--', color = 'green', label = 'Best Epoch')
    ax.legend(loc = 1)
    ax.set_ylim([0.01, 1])

    ax.set_xlabel(xlabel)
    ax.set_ylabel('Accuracy (Fraction)')

    plt.show()

class models:
  def DenseClassifier(hidden_layer_sizes, nn_params, dropout = 0.5):
    model = Sequential()
    model.add(Flatten(input_shape = nn_params['input_shape']))
    for ilayer in hidden_layer_sizes:
      model.add(Dense(ilayer, activation = 'relu'))
      if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(units = nn_params['output_neurons'], activation = nn_params['output_activation']))
    model.compile(loss=nn_params['loss'],
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.95),
                  metrics=['accuracy'])
    return model

  def CNNClassifier(num_hidden_layers, nn_params, dropout = 0.5):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=nn_params['input_shape'], padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(num_hidden_layers-1):
        model.add(Conv2D(32, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(dropout))

    model.add(Dense(units = 64, activation = 'relu'))


    model.add(Dense(units = nn_params['output_neurons'], activation = nn_params['output_activation']))

    # initiate RMSprop optimizer
    opt = tensorflow.keras.optimizers.RMSprop(lr=1e-4)

    # Let's train the model using RMSprop
    model.compile(loss=nn_params['loss'],
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

  def TransferClassifier(name, nn_params, trainable = True):
    expert_dict = {'VGG16': VGG16,
                   'VGG19': VGG19,
                   'ResNet50':ResNet50,
                   'DenseNet121':DenseNet121}

    expert_conv = expert_dict[name](weights = 'imagenet',
                                              include_top = False,
                                              input_shape = nn_params['input_shape'])
    for layer in expert_conv.layers:
      layer.trainable = trainable

    expert_model = Sequential()
    expert_model.add(expert_conv)
    expert_model.add(GlobalAveragePooling2D())

    expert_model.add(Dense(128, activation = 'relu'))
    expert_model.add(Dropout(0.3))

    expert_model.add(Dense(64, activation = 'relu'))

    expert_model.add(Dense(nn_params['output_neurons'], activation = nn_params['output_activation']))

    expert_model.compile(loss = nn_params['loss'],
                  optimizer = optimizers.SGD(lr=1e-4, momentum=0.95),
                  metrics=['accuracy'])

    return expert_model

### defining project variables
# file variables
# image_data_url       = 'https://drive.google.com/uc?id=1qmTuUyn0525-612yS-wkp8gHB72Wv_XP'
# metadata_url         = 'https://drive.google.com/uc?id=1OfKnq3uIT29sXjWSZqOOpceig8Ul24OW'
image_data_path      = './image_data.npy'
metadata_path        = './metadata.csv'
image_shape          = (64, 64, 3)

# neural net parameters
nn_params = {}
nn_params['input_shape']       = image_shape
nn_params['output_neurons']    = 4
nn_params['loss']              = 'categorical_crossentropy'
nn_params['output_activation'] = 'softmax'

### pre-loading all data of interest
_all_data = np.load('image_data.npy')
_metadata = pkg.get_metadata(metadata_path, ['train','test','field'])

### preparing definitions
# downloading and loading data
get_data_split = pkg.get_data_split
get_metadata    = lambda :                 pkg.get_metadata(metadata_path, ['train','test'])
get_train_data  = lambda flatten = False : pkg.get_train_data(flatten = flatten, all_data = _all_data, metadata = _metadata, image_shape = image_shape)
get_test_data   = lambda flatten = False : pkg.get_test_data(flatten = flatten, all_data = _all_data, metadata = _metadata, image_shape = image_shape)
get_field_data  = lambda flatten = False : pkg.get_field_data(flatten = flatten, all_data = _all_data, metadata = _metadata, image_shape = image_shape)

# plotting
plot_image = lambda data, num_ims,figsize=(8,6), labels = [], index = None: helpers.plot_image(data = data, num_ims=num_ims, figsize=figsize,labels = labels, index = index, image_shape = image_shape);
plot_acc       = lambda history: helpers.plot_acc(history)

# querying and combining data
model_to_string        = lambda model: helpers.model_to_string(model)
get_misclassified_data = helpers.get_misclassified_data;
combine_data           = helpers.combine_data;

# models with input parameters
DenseClassifier     = lambda hidden_layer_sizes: models.DenseClassifier(hidden_layer_sizes = hidden_layer_sizes, nn_params = nn_params);
CNNClassifier       = lambda num_hidden_layers: models.CNNClassifier(num_hidden_layers, nn_params = nn_params);
TransferClassifier  = lambda name: models.TransferClassifier(name = name, nn_params = nn_params);

#monitor = ModelCheckpoint('./model.h5', monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
monitor = ModelCheckpoint('./model.h5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')

print("Setup Successful!")

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


AttributeError: `np.sctypes` was removed in the NumPy 2.0 release. Access dtypes explicitly instead.

In [ ]:
import sys
print(sys.version)

In [ ]:
# @title AI prompt cell

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown,clear_output
from google.colab import ai

dropdown = widgets.Dropdown(
    options=[],
    layout={'width': 'auto'}
)

def update_model_list(new_options):
    dropdown.options = new_options
update_model_list(ai.list_models())

text_input = widgets.Textarea(
    placeholder='Ask me anything....',
    layout={'width': 'auto', 'height': '100px'},
)

button = widgets.Button(
    description='Submit Text',
    disabled=False,
    tooltip='Click to submit the text',
    icon='check'
)

output_area = widgets.Output(
     layout={'width': 'auto', 'max_height': '300px','overflow_y': 'scroll'}
)

def on_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=False)
        accumulated_content = ""
        for new_chunk in ai.generate_text(prompt=text_input.value, model_name=dropdown.value, stream=True):
            if new_chunk is None:
                continue
            accumulated_content += new_chunk
            clear_output(wait=True)
            display(Markdown(accumulated_content))

button.on_click(on_button_clicked)
vbox = widgets.GridBox([dropdown, text_input, button, output_area])

display(HTML("""
<style>
.widget-dropdown select {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
.widget-textarea textarea {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
</style>
"""))
display(vbox)


In the meantime, **discuss**: How might you want to design a website using our machine learning model? What kind of visuals or graphs would be useful to your user?

## Load Your Models into Streamlit

Let's load the models we've trained so far into Streamlit! We can do this by using the tensorflow library. If you open up the files tab (the folder icon on the left sidebar), you can see that we have a `.h5` tensor files downloaded that stores our trained parameters for each of our expert models from Notebook 2 (ResNet50, VGG16, VGG19, DenseNet121). Feel free to also upload your own saved model from any of the previous notebooks (`model.h5`) to try using!

Here is a reference on how to save and load sklearn and tensorflow models!

For `sklearn`:
```python
from joblib import dump, load

# ====== Save model ========
dump(model, 'filename.joblib')

# ====== Load model ========
clf = load('filename.joblib')
```

For `tensorflow`:
```python
import tensorflow as tf
from tf.keras.models import load_model

# ====== Save model ========
model.save("filename.h5")

# ====== Load model ========
tensorflow.keras.models.load_model("filename.h5")
```

Our model today is going to be an sklearn model! Let's load in the model we created last time:

In [ ]:
file_path = 'vgg16.h5'
model = load_model(file_path)

When dealing with Streamlit and web development, we begin to work with **file management**. In Python web development, such as Flask and Streamlit, we use a central Python file to launch our app. In this case, we will use `app.py`.
```
app.py
```

Using Streamlit, we can build a simple webapp like so:

```
%%writefile app.py
```
The `%%writefile` command writes to a file (and creates one if it doesn't already exist!). Everything that follows the rest of this block of code will be written to `app.py`. <br> In this case, we use this command to create our `app.py` file, which you can check in the left sidebar.

In [ ]:
%%writefile app.py
import streamlit as st
st.title('Hello World')

To initiate our `app.py`, we employ both Streamlit and ngrok in tandem. Ngrok serves as a hosting service that enables us to establish secure tunnels to our localhost, making our local server accessible over the internet. Streamlit, on the other hand, is a powerful library designed to turn Python scripts into interactive web applications easily. By integrating Streamlit with ngrok, we create a bridge that connects our Python application to the web, allowing external access.

<br> To get access to our Ngrok server, we need to sign up on their website and get a unique **authentication token**.

<font color=SlateGrey><h2><b>
Use [these](https://drive.google.com/file/d/12zwuOuKh91VSHIHS-6S4ADF4HLC2wKJq/view?usp=sharing) instructions to create a ngrok account and get your authtoken!
</b></h2></font>

<font color=DarkGray><h3><b>
Paste your authtoken below next to `!ngrok authtoken`!
</b></h3></font>

In [ ]:
!ngrok authtoken 2iTDRG67TyKZEUZMQCVYiV9ZcLY_3AumQRAPhn9vgJuSRYL8x

Now, we can launch our website through the `launch_website()` function we have written below. We can connect our ngrok token by building a 'tunnel' to our Streamlit code. However, with the free plan, we cannot have more than one tunnel to the server.
```
if ngrok.get_tunnels():
    ngrok.kill()
```
ensures that more than one instance of your website is not running at once! (i.e., If there is an existing 'tunnel' already running, it will 'kill' it).

In [ ]:
def launch_website():
  print ("Click this link to try your web app:")
  if ngrok.get_tunnels():
    ngrok.kill()
  tunnel = ngrok.connect() # The URL to connect to
  print (tunnel.public_url)
  !streamlit run --server.port 80 app.py >/dev/null # Connect to the URL through Port 80 (>/dev/null hides outputs)

Run the following code now to use our function. Click on the first link, and hit the `Visit Site` button to view your site!

In [ ]:
launch_website()

Now the question remains, how do we connect our computer vision models to our website?

### Let's review how we might want to predict on an image:

First, we'd like to preprocess our image by resizing it to a constant fixed size of 64x64 for our model to take in. Here are examples of how you might do this:
```python
image.resize((x, y))
```
Afterwards, you'd want to transform our image into an array - a numerical format where each number in the array represents the intensity of a pixel.
```python
img_to_array(image)
```
And finally, you need to expand the dimensions of the model.
```python
np.expand_dims(image_array, axis=0)
```
Most deep learning models are designed to make predictions on batches of data rather than a single sample. When you want to predict the class of a single image, you still need to make it look like a batch, but with size 1. Expanding the dimensions of the array along `axis=0` effectively adds a new dimension at the start of the array, turning the shape of the array from `(height, width, channels)` to `(1, height, width, channels)`.
<br>

Now, try putting this all together and make predictions!

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import numpy as np

def predict_image(img):
    # Preprocess the image
    img = img.resize((64, 64))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])

    # Fill in Class Labels (i.e., what are our four classes?)
    class_labels = ['Attentive', 'DrinkingCoffee', 'UsingRadio', 'UsingMirror' ] # FILL THE LIST IN\
    return class_labels[predicted_class]

You can now test this out by uploading an image via the left sidebar to Colab's files of a distracted (or attentive) driver and get its image path to make a prediction! Feel free to Google an image, or use this example image to test this out! (right-click the image and click 'Save As...', and upload to the file explorer)

![](https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Driver%20Distraction%20Detection/distracted_driver.jpg)

In [ ]:
image = Image.open('/content/distracteddriving.jpg').convert('RGB')
print(predict_image(image))

## Now, let's put this all together!
To create a button with Streamlit to upload our images, we can use
```python
uploaded_file = st.file_uploader("Prompt", type=["jpg", "png"])
```

Fill out the rest of this code to build a simple webapp to predict off uploaded images!

*Tip*: When we add in the `%%writefile` magic command, Colab removes all syntax highlighting, which makes the code a lot harder to read! If you'd like, you can comment out that `%%writefile` line by adding a `#` at the beginning to restore the syntax highlighting as you edit. Make sure you remove that `#` before you run this, so that your `app.py` file is actually overwritten!

In [ ]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import numpy as np

# Load the pre-trained model
model = load_model('vgg16.h5')

def predict_image(img):
    img = img.resize((64, 64))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)

    # Fill in Class Labels (i.e., what are our four classes?)
    class_labels = ['Attentive', 'DrinkingCoffee', 'UsingRadio', 'UsingMirror' ] # FILL THE LIST IN\
    return class_labels[predicted_class]

st.title("Distracted Driving is bad")

# Create your upload image button
uploaded_file = uploaded_file = st.file_uploader("Prompt", type=["jpg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption='Uploaded Image.', use_column_width=True)

    # Make your prediction here
    predictions = predict_image(image)
    st.write(predictions)

While writing your prediction code, recall that our class labels are as follows:
```
['Attentive', 'Distracted - Drinking Coffee', 'Distracted - Using Mirror', 'Distracted - Using Radio']
```

In [ ]:
launch_website()

## (Challenge) Adding a Saliency Map

As before, we can import our saliency map with a similar function, but modified for Streamlit. To do this, we can create a new file, called `utils.py`. We've included additional functions, such as `saliency_map_to_rgb` and `overlay_saliency_map` to overlay our map on our image!

Adjust the alpha value in `overlay_saliency_map` to change the opacity of the saliency map overlay!

In [ ]:
#@title Saliency Map
%%writefile utils.py

import numpy as np
import matplotlib.pyplot as plt  # for colormap
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils import normalize
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

def get_saliency_map(model, input_image, class_index):
    """
    generate saliency map for a given class and image
    """
    # define loss function
    score = CategoricalScore(class_index)

    # create saliency object
    saliency = Saliency(model,
                        model_modifier=ReplaceToLinear(),
                        clone=True)

    # generate saliency map
    saliency_map = saliency(score, input_image)
    saliency_map = normalize(saliency_map)

    return saliency_map

def saliency_to_rgb(saliency_map):
    """
    convert saliency map to rgb using blue-red color scheme
    """
    # normalize the saliency map for better visualization
    saliency_map_normalized = (saliency_map - np.min(saliency_map)) / (np.max(saliency_map) - np.min(saliency_map))

    saliency_map_colored = plt.get_cmap('jet')(saliency_map_normalized)[:, :, :3]  # exclude alpha channel
    saliency_map_colored = (saliency_map_colored * 255).astype(np.uint8)  # convert to uint8 for displaying as an image

    return saliency_map_colored

def overlay_saliency_map(image, saliency_map, alpha=0.8):
    """
    overlay saliency map on image
    """
    image = image[0].astype(np.uint8)
    saliency_map = saliency_map[0]
    saliency_rgb = saliency_to_rgb(saliency_map)
    return (image * (1 - alpha) + saliency_rgb * alpha).astype(np.uint8)

In Python, you can use functions from other files by importing them! For example, if we had a file called `helper.py` where we wanted to use their `hello_world()` function:
```python
%%writefile helper.py

def hello_world():
    print("hello world!")
```
We can add the following to our app.py file to ***import*** and use it!
```python
%%writefile app.py
from helper import hello_world

hello_world()
```
Now, try importing our saliency map function to display saliency maps! <br> **Note that you will have to resize and expand your dimensions before inputting your image into the saliency_map function!**

In [ ]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import numpy as np
from utils import *

# Load the pre-trained model
model = load_model('vgg16.h5')

def predict_image(img):
    img = img.resize((64, 64))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])

    # Fill in Class Labels (i.e., what are our four classes?)
    class_labels = ['Attentive', 'DrinkingCoffee', 'UsingRadio', 'UsingMirror' ] # FILL THE LIST IN\
    return class_labels[predicted_class]

st.title("Distracted Driving is bad")

# Create your upload image button
uploaded_file = uploaded_file = st.file_uploader("Prompt", type=["jpg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption='Uploaded Image.', use_column_width=True)

    # Make your prediction here
    predictions = predict_image(image)
    st.write(predictions)

    img = image.resize((64, 64))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])

    saliency_map = get_saliency_map(model, img_array, predicted_class)
    overlay = overlay_saliency_map(img_array, saliency_map, alpha=0.6)
    st.image(overlay, caption='Uploaded Image.', use_column_width=True)

In [ ]:
launch_website()